# Strategy analysis example

Debugging a strategy can be time-consuming. Freqtrade offers helper functions to visualize raw data.
The following assumes you work with SampleStrategy, data for 5m timeframe from Binance and have downloaded them into the data directory in the default location.
Please follow the [documentation](https://www.freqtrade.io/en/stable/data-download/) for more details.

## Setup

### Change Working directory to repository root

In [2]:
import os
from pathlib import Path

# Change directory
# Modify this cell to insure that the output shows the correct path.
# Define all paths relative to the project root shown in the cell output
project_root = "somedir/freqtrade"
i=0
try:
    os.chdirdir(project_root)
    assert Path('LICENSE').is_file()
except:
    while i<4 and (not Path('LICENSE').is_file()):
        os.chdir(Path(Path.cwd(), '../'))
        i+=1
    project_root = Path.cwd()
print(Path.cwd())

/Users/cemck/Documents/Trading/freqtrade


### Configure Freqtrade environment

In [108]:
from freqtrade.configuration import Configuration

# Customize these according to your needs.

# Initialize empty configuration object
#config = Configuration.from_files([])
# Optionally (recommended), use existing configuration file
config = Configuration.from_files([str(project_root)+"/config.json"])

# Define some constants
config["timeframe"] = "1m"
# Name of the strategy class
config["strategy"] = "SampleStrategy"
# Location of the data
data_location = config["datadir"]
print(data_location)
# Pair to analyze - Only use one pair here
pair = "BTC/USDT:USDT"

/Users/cemck/Documents/Trading/freqtrade/user_data/data/bitget


In [109]:
# Load data using values set above
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType

candles = load_pair_history(datadir=data_location,
                            timeframe=config["timeframe"],
                            pair=pair,
                            data_format = "json",  # Make sure to update this to your data
                            candle_type=CandleType.FUTURES,
                            )

# Confirm success
print(f"Loaded {len(candles)} rows of data for {pair} from {data_location}")
candles.tail()

Loaded 43529 rows of data for BTC/USDT:USDT from /Users/cemck/Documents/Trading/freqtrade/user_data/data/bitget


,date,open,high,low,close,volume
43524,2023-06-23 22:39:00+00:00,30608.5,30632.5,30608.0,30622.5,81.068
43525,2023-06-23 22:40:00+00:00,30622.5,30629.5,30618.0,30629.0,35.923
43526,2023-06-23 22:41:00+00:00,30629.0,30629.5,30619.5,30619.5,26.523
43527,2023-06-23 22:42:00+00:00,30619.5,30639.0,30608.5,30631.5,50.062
43528,2023-06-23 22:43:00+00:00,30631.5,30641.5,30629.5,30630.0,48.534


#### OBV RSI

In [110]:
import numpy as np
import talib.abstract as ta
from technical import qtpylib
import pandas as pd
import pandas_ta as pandas_ta
import talib as ta

rsi_length = 5
pd.set_option("display.precision", 3)

# https://stackoverflow.com/a/66827219
candles['obv'] = (np.sign(candles['close'].diff()) * candles['volume']).fillna(0).cumsum()

candles['change_1'] = candles['close'].diff()
candles['obv_change'] = candles['obv'].diff()
# obv = np.where(candles['change_1'] > 0, candles['volume'], np.where(candles['change_1'] < 0, -candles['volume'], 0))

#candles['obv'] = ta.OBV(candles)
#candles['obv_rsi'] = ta.RSI(candles['obv'], length=rsi_length)
#num_rows, num_cols = candles.shape
#print(num_rows, num_cols)

def cum(source):
    for i in range(len(source)):
        _cum += source[i]
        return _cum

# def obv(df) -> list:
#     obv_list = []
#     for row in df.iterrows():
#         _obv = None
#         # print(row)
#         vol = row[1]['volume']
#         change_1 = row[1]['change_1']
#         print("vol: " + str(vol))
#         print("change_1: " + str(change_1))
#         if change_1 > 0:
#             _obv = vol
#         elif change_1 < 0:
#             _obv = -vol
#         else:
#             _obv = 0*vol
#         print("obv :" + str(_obv))
#         obv_list.append(_obv)
#     return obv_list

# obv_list = obv(df)
# print(obv_list)
# li = np.cumsum(obv_list)
# print(li)
# candles.head()
# df2 = candles.tail(rsi_length)

# print(df2['obv_change'])

# print(df2)
# c = candles['obv_change'].tail(rsi_length)
# print(c)

# print(np.amax(c))
candles['max_obv_change'] = candles['obv_change']
candles['min_obv_change'] = candles['obv_change']

candles['max_obv_change'] = candles['max_obv_change'].clip(lower=0)
candles['min_obv_change'] = candles['min_obv_change'].clip(upper=0).abs()

def smoothing(prices: pd.Series, smoothing_type, smoothing_period) -> pd.Series:
        if smoothing_type == 'sma':
            return ta.SMA(prices, smoothing_period)
        elif smoothing_type == 'ema':
            return ta.EMA(prices, smoothing_period)
        elif smoothing_type == 'rma':
            return pandas_ta.rma(prices, smoothing_period)
        elif smoothing_type is None:
            return prices
    
def runRMA(df, column) -> pd.Series:
    atr = smoothing(df[column], 'rma', rsi_length)
    return atr

candles['up'] = runRMA(candles, 'max_obv_change')
candles['down'] = runRMA(candles, 'min_obv_change')
# obv = np.where(candles['change_1'] > 0, candles['volume'], np.where(candles['change_1'] < 0, -candles['volume'], 0))

candles['rsi'] = np.where(candles['down'] == 0, 100, np.where(candles['up'] == 0, 0, 100 - 100/(1 + candles['up']/candles['down'])))

candles.tail()

,date,open,high,low,close,volume,obv,change_1,obv_change,max_obv_change,min_obv_change,up,down,rsi
43524,2023-06-23 22:39:00+00:00,30608.5,30632.5,30608.0,30622.5,81.068,39437.333,14.0,81.068,81.068,0.000,53.587,28.668,65.148
43525,2023-06-23 22:40:00+00:00,30622.5,30629.5,30618.0,30629.0,35.923,39473.256,6.5,35.923,35.923,0.000,50.054,22.934,68.578
43526,2023-06-23 22:41:00+00:00,30629.0,30629.5,30619.5,30619.5,26.523,39446.733,-9.5,-26.523,0.000,26.523,40.043,23.652,62.867
43527,2023-06-23 22:42:00+00:00,30619.5,30639.0,30608.5,30631.5,50.062,39496.795,12.0,50.062,50.062,0.000,42.047,18.922,68.965
43528,2023-06-23 22:43:00+00:00,30631.5,30641.5,30629.5,30630.0,48.534,39448.261,-1.5,-48.534,0.000,48.534,33.638,24.844,57.518


#### VWAP Deviation

In [135]:
import math

vwap_length = 60
vwap_source = 'close'


# num_rows, num_cols = dfRange.shape
# print(num_rows, num_cols)

# candles['vwmean'] = (candles[vwap_source] * candles['volume']).rolling(window=vwap_length).sum()/candles['volume'].rolling(window=vwap_length).sum()

# Calculate the rolling volume-weighted mean
rolling_vwmean = candles['close'].rolling(window=60).apply(
    lambda x: np.average(x, weights=candles.loc[x.index, 'volume'])
)

rolling_vwap_std = candles['close'].rolling(window=60).apply(
    lambda x: np.sqrt(np.average((x - np.average(x, weights=candles.loc[x.index, 'volume']))**2, weights=candles.loc[x.index, 'volume']))
)

rolling_vwap_av_std = candles['close'].rolling(window=60).apply(
    lambda x: np.average(np.abs((x - np.average(x, weights=candles.loc[x.index, 'volume']))), weights=candles.loc[x.index, 'volume'])
)

candles['vwmean'] = rolling_vwmean
candles['vwstdev'] = rolling_vwap_std
candles['vwavdev'] = rolling_vwap_av_std

candles['upper_dev'] = candles['vwmean'] + candles['vwstdev'] * 2
candles['lower_dev'] = candles['vwmean'] - candles['vwstdev'] * 2
# candles['vwstdev'] = np.sqrt((candles['volume'] * pow((candles['close'] - candles['vwmean']), 2)).rolling(window=vwap_length).sum()/candles['volume'].rolling(window=vwap_length).sum())
# candles['vwavdev'] = (candles['volume'] * (candles['close'] - candles['vwmean']).abs()).rolling(window=vwap_length).sum()/candles['volume'].rolling(window=vwap_length).sum()

# candles['pow'] = candles['volume'] * np.power((candles[vwap_source] - candles['vwmean']), 2)


candles.tail()

,date,open,high,low,close,volume,obv,change_1,obv_change,max_obv_change,min_obv_change,up,down,rsi,vwmean,vwstdev,vwavdev,upper_dev,lower_dev
43524,2023-06-23 22:39:00+00:00,30608.5,30632.5,30608.0,30622.5,81.068,39437.333,14.0,81.068,81.068,0.000,53.587,28.668,65.148,30595.103,70.595,50.588,30736.293,30453.914
43525,2023-06-23 22:40:00+00:00,30622.5,30629.5,30618.0,30629.0,35.923,39473.256,6.5,35.923,35.923,0.000,50.054,22.934,68.578,30593.079,68.156,49.108,30729.390,30456.767
43526,2023-06-23 22:41:00+00:00,30629.0,30629.5,30619.5,30619.5,26.523,39446.733,-9.5,-26.523,0.000,26.523,40.043,23.652,62.867,30591.775,66.356,48.080,30724.488,30459.062
43527,2023-06-23 22:42:00+00:00,30619.5,30639.0,30608.5,30631.5,50.062,39496.795,12.0,50.062,50.062,0.000,42.047,18.922,68.965,30591.016,64.819,47.262,30720.655,30461.377
43528,2023-06-23 22:43:00+00:00,30631.5,30641.5,30629.5,30630.0,48.534,39448.261,-1.5,-48.534,0.000,48.534,33.638,24.844,57.518,30588.939,61.371,45.563,30711.681,30466.196


## Load and run strategy
* Rerun each time the strategy file is changed

In [ ]:
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver
from freqtrade.data.dataprovider import DataProvider
strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)
strategy.ft_bot_start()

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles, {'pair': pair})
df.tail()

### Display the trade details

* Note that using `data.head()` would also work, however most indicators have some "startup" data at the top of the dataframe.
* Some possible problems
    * Columns with NaN values at the end of the dataframe
    * Columns used in `crossed*()` functions with completely different units
* Comparison with full backtest
    * having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
    * Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  


In [ ]:
# Report results
print(f"Generated {df['enter_long'].sum()} entry signals")
data = df.set_index('date', drop=False)
data.tail()

## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [ ]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

# if backtest_dir points to a directory, it'll automatically load the last backtest file.
backtest_dir = config["user_data_dir"] / "backtest_results"
# backtest_dir can also point to a specific file 
# backtest_dir = config["user_data_dir"] / "backtest_results/backtest-result-2020-07-01_20-04-22.json"

In [ ]:
# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
stats = load_backtest_stats(backtest_dir)

strategy = 'SampleStrategy'
# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this will be reflected here as well.
# Example usages:
print(stats['strategy'][strategy]['results_per_pair'])
# Get pairlist used for this backtest
print(stats['strategy'][strategy]['pairlist'])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats['strategy'][strategy]['market_change'])
# Maximum drawdown ()
print(stats['strategy'][strategy]['max_drawdown'])
# Maximum drawdown start and end
print(stats['strategy'][strategy]['drawdown_start'])
print(stats['strategy'][strategy]['drawdown_end'])


# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats['strategy_comparison'])


In [ ]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["exit_reason"].value_counts()

## Plotting daily profit / equity line

In [ ]:
# Plotting equity line (starting with 0 on day 1 and adding daily profit for each backtested day)

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
import plotly.express as px
import pandas as pd

# strategy = 'SampleStrategy'
# config = Configuration.from_files(["user_data/config.json"])
# backtest_dir = config["user_data_dir"] / "backtest_results"

stats = load_backtest_stats(backtest_dir)
strategy_stats = stats['strategy'][strategy]

dates = []
profits = []
for date_profit in strategy_stats['daily_profit']:
    dates.append(date_profit[0])
    profits.append(date_profit[1])

equity = 0
equity_daily = []
for daily_profit in profits:
    equity_daily.append(equity)
    equity += float(daily_profit)


df = pd.DataFrame({'dates': dates,'equity_daily': equity_daily})

fig = px.line(df, x="dates", y="equity_daily")
fig.show()


### Load live trading results into a pandas dataframe

In case you did already some trading and want to analyze your performance

In [ ]:
from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["exit_reason"].value_counts()

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

`analyze_trade_parallelism()` returns a timeseries dataframe with an "open_trades" column, specifying the number of open trades for each candle.

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '5m')

parallel_trades.plot()

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]

data_red = data['2019-06-01':'2019-06-10']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['sma20', 'ema50', 'ema55'],
                                   indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                  )




In [ ]:
# Show graph inline
# graph.show()

# Render graph in a separate window
graph.show(renderer="browser")


## Plot average profit per trade as distribution graph

In [ ]:
import plotly.figure_factory as ff

hist_data = [trades.profit_ratio]
group_labels = ['profit_ratio']  # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size=0.01)
fig.show()


Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.